In [ ]:
import pandas as pd
df=pd.read_csv('/content/fraudTest.csv')
df.sample(4)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
119,119,2020-06-21 12:51:26,2720433095629877,fraud_Powlowski-Weimann,food_dining,1.92,Mark,Wood,M,854 Walker Dale Suite 488,...,44.0575,-69.9656,3224.0,"Engineer, electronics",1997-08-22,91a54264afc003d048c777e4ad5b1afc,1.371819e+09,44.191357,-70.730185,0.0
8483,8483,2020-06-23 16:28:42,345832460465610,fraud_Adams-Barrows,health_fitness,19.66,Jason,Mcmahon,M,6385 Donald Square Suite 429,...,38.8029,-77.2116,104396.0,Production engineer,1950-11-20,c7919519e14f82b15965238efe0a5003,1.372005e+09,38.965315,-76.567868,0.0
728,728,2020-06-21 16:22:15,3518234918950662,"fraud_Gottlieb, Considine and Schultz",shopping_net,724.14,Barbara,Lowe,F,67210 Lori Creek,...,33.6372,-96.6184,46563.0,Systems developer,1970-09-11,e1d1723770f702944afd2ec020fd9df0,1.371832e+09,34.496590,-96.913335,0.0
5690,5690,2020-06-22 22:00:26,4809701904914,"fraud_Daugherty, Pouros and Beahan",shopping_pos,9.41,Sabrina,Nolan,F,312 Eric Expressway Apt. 014,...,32.4204,-85.0796,59744.0,Chemical engineer,1984-02-07,b4f52174a403ee35c4e53f789915d402,1.371938e+09,31.813589,-85.776518,0.0


In [23]:
import numpy as np
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
print(df.head())
print(df.info())
print(df.describe())

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2020-06-21 12:14:25  2291163933867244   
1           1   2020-06-21 12:14:33  3573030041201292   
2           2   2020-06-21 12:14:53  3598215285024754   
3           3   2020-06-21 12:15:15  3591919803438423   
4           4   2020-06-21 12:15:17  3526826139003047   

                               merchant        category    amt   first  \
0                 fraud_Kirlin and Sons   personal_care   2.86    Jeff   
1                  fraud_Sporer-Keebler   personal_care  29.84  Joanne   
2  fraud_Swaniawski, Nitzsche and Welch  health_fitness  41.28  Ashley   
3                     fraud_Haley Group        misc_pos  60.05   Brian   
4                 fraud_Johnston-Casper          travel   3.19  Nathan   

       last gender                       street  ...      lat      long  \
0   Elliott      M            351 Darlene Green  ...  33.9659  -80.9355   
1  Williams      F             3638 Marsh Union  ...  40.3207 

In [36]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['trans_year'] = df['trans_date_trans_time'].dt.year
df['trans_month'] = df['trans_date_trans_time'].dt.month
df['trans_day'] = df['trans_date_trans_time'].dt.day
df['trans_hour'] = df['trans_date_trans_time'].dt.hour
df['trans_minute'] = df['trans_date_trans_time'].dt.minute
df['trans_second'] = df['trans_date_trans_time'].dt.second
df = df.drop('trans_date_trans_time', axis=1)

In [31]:
target_column = 'is_fraud'  # This column indicates fraudulent transactions

# Check if the target column exists in the dataset
if target_column not in df.columns:
    raise ValueError(f"The target column '{target_column}' does not exist in the dataset.")



In [32]:
f = df.dropna()

# Encode categorical variables
label_encoder = LabelEncoder()
categorical_columns = ['first', 'last', 'gender', 'street', 'job']
for column in categorical_columns:
    if column in df.columns:
        df[column] = label_encoder.fit_transform(df[column])
    else:
        raise ValueError(f"The column '{column}' does not exist in the dataset.")

In [33]:
fraudulent_transactions = df[df[target_column] == 1]
non_fraudulent_transactions = df[df[target_column] == 0]

# Oversample the minority class (fraudulent transactions)
fraudulent_transactions_oversampled = fraudulent_transactions.sample(len(non_fraudulent_transactions), replace=True)

# Combine the oversampled fraudulent transactions with the non-fraudulent transactions
df_balanced = pd.concat([fraudulent_transactions_oversampled, non_fraudulent_transactions])


In [34]:
X = df_balanced.drop(target_column, axis=1)
y = df_balanced[target_column]

In [42]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[numeric_columns])

# Apply PCA for dimensionality reduction
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_scaled)


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log = log_reg.predict(X_test)

In [44]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)


In [45]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [46]:
print("Logistic Regression:")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("Precision:", precision_score(y_test, y_pred_log))
print("Recall:", recall_score(y_test, y_pred_log))
print("F1-score:", f1_score(y_test, y_pred_log))
print("Classification Report:")
print(classification_report(y_test, y_pred_log))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_log))

print("Decision Tree Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt))
print("Recall:", recall_score(y_test, y_pred_dt))
print("F1-score:", f1_score(y_test, y_pred_dt))
print("Classification Report:")
print(classification_report(y_test, y_pred_dt))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))

print("Random Forest Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1-score:", f1_score(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

Logistic Regression:
Accuracy: 0.8721484727413327
Precision: 0.8936111872772141
Recall: 0.8436448356199845
F1-score: 0.8679094540612516
Classification Report:
              precision    recall  f1-score   support

         0.0       0.85      0.90      0.88      3896
         1.0       0.89      0.84      0.87      3863

    accuracy                           0.87      7759
   macro avg       0.87      0.87      0.87      7759
weighted avg       0.87      0.87      0.87      7759

Confusion Matrix:
[[3508  388]
 [ 604 3259]]
Decision Tree Classifier:
Accuracy: 0.9988400567083388
Precision: 0.9976756198347108
Recall: 1.0
F1-score: 0.9988364576599871
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3896
         1.0       1.00      1.00      1.00      3863

    accuracy                           1.00      7759
   macro avg       1.00      1.00      1.00      7759
weighted avg       1.00      1.00      1.00     